In [1]:
from graph_ds import PyH3Graph

In [2]:
graph = PyH3Graph()
graph.create(osm_path="../resources/copenhagen-processed.osm.pbf", gtfs_path="../resources/rejseplanen.zip")

processing osm pbf file: ../resources/copenhagen-processed.osm.pbf
converted OSM file into 446019 edges
osm graph created with 364511 nodes in 10 s
getting GTFS feed from ../resources/rejseplanen.zip
routes: 1631
gtfs graph created with 103525 nodes in 36 s
merged graph created with 465889 nodes in 0 s


In [3]:
graph.astar_path(631049690224658431, 631049690149925887)

([631049690149925887,
  631049690149927935,
  631049690149927423,
  631049690149913599,
  631049690149915647,
  631049690149915135,
  631049690149929983,
  631049690149932031,
  631049690149828095,
  631049690149830143,
  631049690149816319,
  631049690149814783,
  631049690149816831,
  631049690149831679,
  631049690149833727,
  631049690149959167,
  631049690149961215,
  631049690149947391,
  631049690149945855,
  631049690149947903,
  631049690149963775,
  631049690149962239,
  631049690149965311,
  631049690150905855,
  631049690150905343,
  631049690150967807,
  631049690150983679,
  631049690150983167,
  631049690150985215,
  631049690150877183,
  631049690150875647,
  631049690150877695,
  631049690150863871,
  631049690150865919,
  631049690150884863,
  631049690150880255,
  631049690150881279,
  631049690150880767,
  631049690150882815,
  631049690151066623,
  631049690151003647,
  631049690151005695,
  631049690151083007,
  631049690151020031,
  631049690151022079,
  63104969

In [4]:
%%time
dist_result = graph.matrix_bfs(origins=[631049690224658431, 631049690149925887], destinations=[631049690149925887, 631049690224658431])

CPU times: user 170 ms, sys: 115 µs, total: 170 ms
Wall time: 85.1 ms


In [5]:
dist_result

[[7.120895977499999], [5.8967167819999995]]